In [1]:
%reload_ext autoreload
%autoreload 2

In [13]:
from nbc_analysis import get_config, get_test_data, extract_events
from nbc_analysis.utils.debug_utils import runit
import numpy as np

In [17]:
CONFIG_F='default'
config = runit(get_config, overrides={}, config_f=CONFIG_F)
config

>> created example config file '/Users/wmcabee/.config/nbc_analysis/config_example.yaml'
>> Using default config


{'VIDEO_END_BUCKET': 'nbc-event',
 'EVENT_BATCHES_D': '/Users/wmcabee/DATA/NBC2/batches',
 'LIMIT_EVENTS_PER_BATCH': None}

In [20]:
days = get_test_data('days.csv').day.astype(np.str)[:2].tolist()
ret = runit(extract_events, days=days, config_f=CONFIG_F)
#ret
days

>> created example config file '/Users/wmcabee/.config/nbc_analysis/config_example.yaml'
>> Using default config
>> start extract events, config={'VIDEO_END_BUCKET': 'nbc-event', 'EVENT_BATCHES_D': '/Users/wmcabee/DATA/NBC2/batches', 'LIMIT_EVENTS_PER_BATCH': None}


FileNotFoundError: [Errno 2] File b'20190701' does not exist: b'20190701'

In [87]:
# https://nbc-event.s3.amazonaws.com/year%3D2019/month%3D07/event%3Dvideo_end/NBCUniversal_20190701000737029958_74353.txt
prefix='year={year}/month={month}/event=video_end/NBCUniversal_{day}'

prefix.format(**ret)

'year=2019/month=07/event=video_end/NBCUniversal_01'

In [60]:
arrow.utcnow().isoformat()

'2019-09-26T20:57:12.734985+00:00'

In [78]:
day = "20190703"

import re

m = re.match("(?P<year>\d{4})(?P<month>\d{2})(?P<day>\d{2})", day)
m.groupdict()

{'year': '2019', 'month': '07', 'day': '03'}